In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.datasets import mnist
from tensorflow_model_optimization.python.core.keras import metrics
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_schedule as pruning_sched
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper


np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 5.1 MB/s eta 0:00:00


In [ ]:
# Load and preprocess MNIST data

# Load the data and split it between train and test sets
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()
(x_train_c_10, y_train_c_10), (x_test_c_10, y_test_c_10) = cifar10.load_data()
(x_train_c_100, y_train_c_100), (x_test_c_100, y_test_c_100) = cifar100.load_data()

def preprocess(x_train,y_train,x_test,y_test,num_classes):
  # Scale images to the [0, 1] range
  x_train = x_train.astype("float32") / 255
  x_test = x_test.astype("float32") / 255
  # Make sure images have shape (28, 28, 1)
  x_train = np.expand_dims(x_train, -1)
  x_test = np.expand_dims(x_test, -1)
  print("x_train shape:", x_train.shape)
  print(x_train.shape[0], "train samples")
  print(x_test.shape[0], "test samples")
  # convert class vectors to binary class matrices
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)

  return x_train,y_train,x_test,y_test

x_train_mnist, y_train_mnist,x_test_mnist, y_test_mnist = preprocess(x_train_mnist, y_train_mnist,x_test_mnist, y_test_mnist,10)
x_train_c_10, y_train_c_10,x_test_c_10, y_test_c_10 = preprocess(x_train_c_10, y_train_c_10,x_test_c_10, y_test_c_10,10)
x_train_c_100, y_train_c_100,x_test_c_100, y_test_c_100 = preprocess(x_train_c_100, y_train_c_100,x_test_c_100, y_test_c_100,100)



169001437/169001437 [==============================] - 4s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples
x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


# Implementing L1 Prune in Keras

In [ ]:



def _compute_nparams_toprune(amount, tensor_size):
    return round(amount * tensor_size)


def Ln_mask(x,amount):
    if amount == 0:
        return np.ones(x.shape)
    mask = np.ones(x.shape)
    norm = np.linalg.norm(x, ord=1, axis=-1)
    nparams_to_prune = _compute_nparams_toprune(amount, x.shape[0])
    if nparams_to_prune == x.shape[0]:
        nparams_to_prune = nparams_to_prune - 1
    topk = np.argpartition(norm, nparams_to_prune)
    mask[topk[:nparams_to_prune]] = 0
    return mask


def prune(model, amount):
    weights = model.get_weights()
    shapes = []
    for w in weights:
        shapes.append(w.shape)
    flatten_weights = []
    for w in weights:
        flatten_weights.extend(w.flatten())
    flatten_weights = np.array(flatten_weights).reshape(-1, 1)
    mask = Ln_mask(x=flatten_weights, amount=amount)
    new_weights = flatten_weights * mask
    new_weights = new_weights.flatten()
    new_model_weights = []
    elm_prod = []
    for i in shapes:
        elm_prod.append(np.prod(i))

    for i in range(len(shapes)):
        if i == 0:
            new_model_weights.append(
                np.array(new_weights[0 : np.prod(shapes[i])]).reshape(shapes[i])
            )
        else:
            new_model_weights.append(
                np.array(
                    new_weights[np.sum(elm_prod[:i]) : np.sum(elm_prod[: i + 1])]
                ).reshape(shapes[i])
            )
    model.set_weights(new_model_weights)
    return model

In [ ]:
def save_function(model,test_data,filepath):
   results = model.predict(test_data)
   softmax_df = pd.DataFrame(results)
   filename = filepath+'25_softmax.csv'
   softmax_df.to_csv(filename,index=False)

In [ ]:
def prune_process(path,x_test,y_test,directory):
  pruned_amount = 0.1
  print("Orginal Model Accuracy")
  model = tf.keras.models.load_model(path)
  print(model.evaluate(x_test,y_test))
  for i in range(1,6):
    if i>1:
      pruned_amount = round((pruned_amount+(0.2)),3)
    pruned_percentage = 100*pruned_amount
    model = tf.keras.models.load_model(path)
    model_pruned = prune(model,pruned_amount)
    model_pruned.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print("Pruned at:",pruned_amount)
    print(model_pruned.evaluate(x_test,y_test))
    save_function(model_pruned,x_test,directory+str(round(pruned_percentage))+'_')

# LeNet MNIST Prune

In [ ]:
!mkdir 'MNIST_pruned'

In [ ]:
path = '/content/baseMNIST/LeNetMNIST_final.tf'
directory = 'MNIST_pruned/'
prune_process(path,x_test_mnist,y_test_mnist,directory)

Orginal Model Accuracy
313/313 [==============================] - 1s 3ms/step - loss: 0.0454 - accuracy: 0.9916
[0.04537806287407875, 0.991599977016449]
Pruned at: 0.1
313/313 [==============================] - 1s 3ms/step - loss: 0.0452 - accuracy: 0.9916
[0.045227717608213425, 0.991599977016449]
313/313 [==============================] - 1s 2ms/step
Pruned at: 0.3
313/313 [==============================] - 1s 4ms/step - loss: 0.0449 - accuracy: 0.9915
[0.044862985610961914, 0.9915000200271606]
313/313 [==============================] - 1s 2ms/step
Pruned at: 0.5
313/313 [==============================] - 1s 3ms/step - loss: 0.0414 - accuracy: 0.9913
[0.041362494230270386, 0.9912999868392944]
313/313 [==============================] - 1s 2ms/step
Pruned at: 0.7
313/313 [==============================] - 1s 3ms/step - loss: 0.0660 - accuracy: 0.9819
[0.06597433239221573, 0.9818999767303467]
313/313 [==============================] - 1s 2ms/step
Pruned at: 0.9
313/313 [=================

#LeNet CIFAR10

In [ ]:
!mkdir 'CIFAR10_pruned'

In [ ]:
path = '/content/baseC10/LeNetCIFAR10_final.tf'
directory = 'CIFAR10_pruned/'
prune_process(path,x_test_c_10,y_test_c_10,directory)

Orginal Model Accuracy
313/313 [==============================] - 1s 3ms/step - loss: 2.4273 - accuracy: 0.6339
[2.427269220352173, 0.633899986743927]
Pruned at: 0.1
313/313 [==============================] - 1s 3ms/step - loss: 2.4208 - accuracy: 0.6355
[2.4208269119262695, 0.6355000138282776]
313/313 [==============================] - 1s 3ms/step
Pruned at: 0.3
313/313 [==============================] - 1s 3ms/step - loss: 2.4524 - accuracy: 0.6301
[2.452402114868164, 0.6301000118255615]
313/313 [==============================] - 1s 2ms/step
Pruned at: 0.5
313/313 [==============================] - 1s 3ms/step - loss: 2.4294 - accuracy: 0.6152
[2.429434061050415, 0.6151999831199646]
313/313 [==============================] - 1s 2ms/step
Pruned at: 0.7
313/313 [==============================] - 1s 3ms/step - loss: 2.6455 - accuracy: 0.5416
[2.645512819290161, 0.5415999889373779]
313/313 [==============================] - 1s 2ms/step
Pruned at: 0.9
313/313 [============================

# LeNet CIFAR100

In [ ]:
!mkdir 'CIFAR100_pruned'

In [ ]:
path = '/content/baseC100/LeNetCIFAR100_final.tf'
directory = 'CIFAR100_pruned/'
prune_process(path,x_test_c_100,y_test_c_100,directory)

Orginal Model Accuracy
313/313 [==============================] - 1s 3ms/step - loss: 3.9201 - accuracy: 0.3268
[3.9201202392578125, 0.32679998874664307]
Pruned at: 0.1
313/313 [==============================] - 1s 3ms/step - loss: 3.9214 - accuracy: 0.3272
[3.921401262283325, 0.3271999955177307]
313/313 [==============================] - 1s 3ms/step
Pruned at: 0.3
313/313 [==============================] - 1s 3ms/step - loss: 3.9156 - accuracy: 0.3245
[3.9156394004821777, 0.3244999945163727]
313/313 [==============================] - 1s 2ms/step
Pruned at: 0.5
313/313 [==============================] - 1s 3ms/step - loss: 3.9938 - accuracy: 0.3137
[3.993757486343384, 0.31369999051094055]
313/313 [==============================] - 1s 2ms/step
Pruned at: 0.7
313/313 [==============================] - 1s 4ms/step - loss: 4.7243 - accuracy: 0.2212
[4.7243194580078125, 0.22120000422000885]
313/313 [==============================] - 1s 3ms/step
Pruned at: 0.9
313/313 [======================

In [ ]:
shutil.make_archive("CIFAR100_pruned", 'zip', "CIFAR100_pruned")
shutil.make_archive("CIFAR10_pruned", 'zip', "CIFAR10_pruned")
shutil.make_archive("MNIST_pruned", 'zip', "MNIST_pruned")


'/content/MNIST_pruned.zip'

# ResNet 50 CIFAR 10

In [ ]:
!mkdir 'CIFAR10_pruned'

In [ ]:
path = '/content/baseResC10/resnetCIFAR10_final.tf'
directory = 'CIFAR10_pruned/'
prune_process(path,x_test_c_10,y_test_c_10,directory)

Orginal Model Accuracy
313/313 [==============================] - 6s 13ms/step - loss: 1.7435 - accuracy: 0.6738
[1.7434873580932617, 0.673799991607666]
Pruned at: 0.1
313/313 [==============================] - 7s 12ms/step - loss: 1.7450 - accuracy: 0.6733
[1.7450107336044312, 0.67330002784729]
313/313 [==============================] - 4s 9ms/step
Pruned at: 0.3
313/313 [==============================] - 6s 13ms/step - loss: 1.7134 - accuracy: 0.6770
[1.7133902311325073, 0.6769999861717224]
313/313 [==============================] - 4s 9ms/step
Pruned at: 0.5
313/313 [==============================] - 6s 14ms/step - loss: 1.7194 - accuracy: 0.6736
[1.7193787097930908, 0.6736000180244446]
313/313 [==============================] - 4s 9ms/step
Pruned at: 0.7
313/313 [==============================] - 6s 15ms/step - loss: 1.6837 - accuracy: 0.6711
[1.6836689710617065, 0.6711000204086304]
313/313 [==============================] - 4s 9ms/step
Pruned at: 0.9
313/313 [=====================

In [ ]:
shutil.make_archive("CIFAR10_pruned", 'zip', "CIFAR10_pruned")

'/content/CIFAR10_pruned.zip'

# ResNet 50 CIFAR 100

In [ ]:
!mkdir 'CIFAR100_pruned'

In [ ]:
path = '/content/baseResC100/resnetCIFAR100_final.tf'
directory = 'CIFAR100_pruned/'
prune_process(path,x_test_c_100,y_test_c_100,directory)

Orginal Model Accuracy
313/313 [==============================] - 5s 12ms/step - loss: 3.0026 - accuracy: 0.3792
[3.00264573097229, 0.3792000114917755]
Pruned at: 0.1
313/313 [==============================] - 6s 12ms/step - loss: 3.0042 - accuracy: 0.3794
[3.0041580200195312, 0.37940001487731934]
313/313 [==============================] - 5s 11ms/step
Pruned at: 0.3
313/313 [==============================] - 5s 12ms/step - loss: 3.0036 - accuracy: 0.3786
[3.003631830215454, 0.37860000133514404]
313/313 [==============================] - 4s 9ms/step
Pruned at: 0.5
313/313 [==============================] - 6s 14ms/step - loss: 3.0090 - accuracy: 0.3737
[3.0090384483337402, 0.37369999289512634]
313/313 [==============================] - 4s 9ms/step
Pruned at: 0.7
313/313 [==============================] - 6s 12ms/step - loss: 4.9498 - accuracy: 0.0115
[4.949783802032471, 0.011500000022351742]
313/313 [==============================] - 3s 8ms/step
Pruned at: 0.9
313/313 [================

In [ ]:
shutil.make_archive("CIFAR100_pruned", 'zip', "CIFAR100_pruned")

'/content/CIFAR100_pruned.zip'

# MobileNet CIFAR10

In [ ]:
!mkdir 'mobilenetCIFAR10'
path = '/content/mobileNetCifar10.tf'
directory = 'mobilenetCIFAR10/'
prune_process(path,x_test_c_10,y_test_c_10,directory)

Orginal Model Accuracy
313/313 [==============================] - 6s 9ms/step - loss: 1.4095 - accuracy: 0.6871
[1.4094783067703247, 0.6870999932289124]
Pruned at: 0.1
313/313 [==============================] - 3s 6ms/step - loss: 4.5634 - accuracy: 0.5714
[4.563412666320801, 0.571399986743927]
313/313 [==============================] - 2s 4ms/step
Pruned at: 0.3
313/313 [==============================] - 3s 6ms/step - loss: 24.5981 - accuracy: 0.1123
[24.598072052001953, 0.11230000108480453]
313/313 [==============================] - 2s 4ms/step
Pruned at: 0.5
313/313 [==============================] - 3s 6ms/step - loss: 23.8613 - accuracy: 0.1001
[23.861291885375977, 0.10010000318288803]
313/313 [==============================] - 2s 4ms/step
Pruned at: 0.7
313/313 [==============================] - 3s 6ms/step - loss: 7.9449 - accuracy: 0.1110
[7.944864273071289, 0.11100000143051147]
313/313 [==============================] - 2s 4ms/step
Pruned at: 0.9
313/313 [=====================

In [ ]:
shutil.make_archive("mobilenetCIFAR10", 'zip', "mobilenetCIFAR10")

'/content/mobilenetCIFAR10.zip'

# MobileNet CIFAR100

In [ ]:
!mkdir 'mobilenetCIFAR100'
path = '/content/mobileNetCifar100.tf'
directory = 'mobilenetCIFAR100/'
prune_process(path,x_test_c_100,y_test_c_100,directory)

Orginal Model Accuracy
313/313 [==============================] - 2s 6ms/step - loss: 4.0083 - accuracy: 0.2912
[4.008255481719971, 0.29120001196861267]
Pruned at: 0.1
313/313 [==============================] - 3s 6ms/step - loss: 9.4435 - accuracy: 0.1857
[9.443548202514648, 0.18569999933242798]
313/313 [==============================] - 2s 4ms/step
Pruned at: 0.3
313/313 [==============================] - 4s 6ms/step - loss: 35.4507 - accuracy: 0.0124
[35.4506950378418, 0.012400000356137753]
313/313 [==============================] - 2s 4ms/step
Pruned at: 0.5
313/313 [==============================] - 3s 6ms/step - loss: 38.0786 - accuracy: 0.0117
[38.07858657836914, 0.011699999682605267]
313/313 [==============================] - 2s 4ms/step
Pruned at: 0.7
313/313 [==============================] - 4s 6ms/step - loss: 25.5027 - accuracy: 0.0104
[25.502702713012695, 0.010400000028312206]
313/313 [==============================] - 2s 4ms/step
Pruned at: 0.9
313/313 [=================

In [ ]:
shutil.make_archive("mobilenetCIFAR100", 'zip', "mobilenetCIFAR100")

'/content/mobilenetCIFAR100.zip'